<a href="https://colab.research.google.com/github/JacksonLeb/tensorflow_learning/blob/main/08_rnn_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#introduction to NLP fundamentals in Tf

##Getting info from natural language (sequences or speech)
##NLP can also be called seq2seq problems

###Checking for GPU


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-28a7f458-dd56-ff64-a1e1-24a761ea6b72)


In [2]:
##Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-25 22:27:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-25 22:27:28 (109 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
##Get a text dataset
###Using the Kaggle Intro to NLP dataset, samples of tweets labeled hazard or not
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-07-25 22:27:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.96.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-07-25 22:27:31 (114 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

##Visualizing a text dataset

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [9]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [10]:
#shuffle trianing df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
#test df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
#examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
#total num of samples
len(train_df), len(test_df)

(7613, 3263)

In [14]:
#visualizing random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random index
for row in train_df_shuffled[["text", "target"]][random_index:random_index +5 ].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}")

Target: 0 (not real disaster)
Text:
@ColoicCarnality You were bleeding my instincts kicked in. *She looks away and scratches the back of her head*
Target: 1 (real disaster
Text:
into burning fucking buildings (2/2)
Target: 1 (real disaster
Text:
#?? #???? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ...  http://t.co/hHWv0EUDFv
Target: 0 (not real disaster)
Text:
To All The Meat-Loving Feminists Of The World Riot Grill Has Arrived: Pop quiz! Which do you prefer: fem... http://t.co/KmndkFa7me #art
Target: 1 (real disaster
Text:
WACKOS like #MicheleBachman predict the WORLD will SOON be OBLITERATED by a burning firey INFERNO but cant accept #GlobalWarming!! HELLO!!!


In [15]:
#splittting data into training and validation
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=30)

In [16]:
#checking lengths of each data set
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [17]:
#check the first 10 sentences
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [18]:
val_sentences[:10]

array(['Blaze is my bro http://t.co/UdKeSJ01mL',
       'I was taught at school in the 1970s that piracy slavery and suicide-bombing were purely historical. No one then expected them to re-occur',
       'http://t.co/FueRk0gWui Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/Mv7GgGlmVc',
       "I'm a friggin wreck destiel sucks (read the vine description)\nhttps://t.co/MKX6Ux4OZt",
       'Dr. Bengston on #wildfire management: \x89ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.\x89Û\x9d #smem',
       '@ameenshaikh3 sir i just only wanted to make a point about @sureshpprabhu you made and said he is lying about bridge collapse.',
       'Wow! I just won this for free The Hobbit: Desolation of Smaug UV digital download code *GIN 9 http://t.co/MjFdCrjs8j #listia',
       'Photoset: hakogaku: ?åÊI am a kurd. i was born on a battlefield. raised on a battlefield. gunfire sirens... http://t.co/obp595W7tm',
       'Wash

## Converting text into numbers

#Tokenization and Embedding
##Tokenization: Each word/character is set to a number/one-hot encoding
##Embedding: Each word/character is set to a vector, with user-defined size, allowing for information on the relationship between tokens/words/characters. Additionally, can limit size and can be learned (treated as a weight matrix)


In [19]:
#Text Vectorization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

#Use default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long each batch of a setence should be
                                    pad_to_max_tokens=False)


In [20]:
#find average number of tokens (words) in the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
#set up text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocabulary
max_length = 15 #max length of a tweet that our models sees

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=max_length, #how long each batch of a setence should be
                                    pad_to_max_tokens=True)

In [22]:
#adapting text_vectorizer to training data
text_vectorizer.adapt(train_sentences) #adapt to training data, very important

In [23]:
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [24]:
#Create sample setnces and tokenize it
sample_sentence = "A tornado destroyed my house"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  3, 595, 346,  13, 311,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
#Choose a random sentence from the triaing dataset and vectorize it
random_sentence = random.choice(train_sentences)
print(f"original sentence: {random_sentence}\n vectorized sentence: {text_vectorizer([random_sentence])}")

original sentence: @PipRhys I predict a riot.
 vectorized sentence: [[9786    8 2793    3  480    0    0    0    0    0    0    0    0    0
     0]]


In [26]:
#number of unique words found in train_sentences
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Num of words: {len(words_in_vocab)}")
top_5_words, bottom_5_words

Num of words: 10000


(['', '[UNK]', 'the', 'a', 'in'],
 ['panda', 'panama', 'pampalmater', 'palmoil', 'palmer'])

### Creating an embedding using an embedding layer


In [27]:
#using tensorflow.keras.layeres.Embedding
##Parameters: input_dim, output_dim, input_length
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [28]:
#get a random sentence
random_sentence = random.choice(train_sentences)
print(f"random sentence: {random_sentence}")
print(f"embedded:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

random sentence: Fear Factory - Cars (Official MusicVideo) http://t.co/UUzaUMdObc
embedded:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03630776, -0.03820932, -0.00445824, ..., -0.04886023,
         -0.02145875,  0.04040292],
        [-0.01063418,  0.00336517,  0.01590541, ..., -0.047716  ,
         -0.03535693, -0.03974954],
        [-0.01511597,  0.02685222,  0.00669716, ...,  0.03934802,
          0.03094483, -0.02585809],
        ...,
        [-0.01007362, -0.0152952 , -0.00491195, ..., -0.0267398 ,
          0.01454201, -0.00249864],
        [-0.01007362, -0.0152952 , -0.00491195, ..., -0.0267398 ,
          0.01454201, -0.00249864],
        [-0.01007362, -0.0152952 , -0.00491195, ..., -0.0267398 ,
          0.01454201, -0.00249864]]], dtype=float32)>

In [29]:
sample_embed[0][0], sample_embed[0][0].shape, 

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03630776, -0.03820932, -0.00445824,  0.00122439, -0.03274621,
         0.03877516,  0.02226405,  0.03202726,  0.02943074,  0.04005379,
         0.03334017, -0.03114922, -0.01374232, -0.01096838, -0.02020562,
        -0.03324328, -0.03869898, -0.02142135, -0.0289169 ,  0.04141949,
        -0.01022698, -0.02106804, -0.00709133, -0.04640056, -0.0066442 ,
         0.01029294, -0.0002417 ,  0.02963665, -0.03526785, -0.01382786,
        -0.02770214,  0.00579495, -0.03142303, -0.01239208, -0.02850951,
         0.01319606,  0.02045951,  0.04064058,  0.00234356,  0.01939703,
        -0.00700695, -0.01180419,  0.03196505,  0.01736821,  0.03173614,
         0.03339514, -0.04529649, -0.03753884, -0.01445227, -0.04492917,
        -0.01459052,  0.04039009,  0.02911403, -0.02472272, -0.01846645,
        -0.03449081,  0.00277895, -0.03346279, -0.02423668,  0.02207308,
        -0.00086076, -0.00553368,  0.00775637,  0.00055003,  0.01623397,
  

##Modelling a text dataset (running series of experiments)
###1. Naive bayes with Tf-IDF encoder (BASELINE)
###2. Feed-forward neural network 
###3. LSTM
###4. GRU
###5. Bidirectiona;-LSTM
###6. 1D CNN
###7. TFHub pretrianed feature extractor
###8. TFHub pretrained with 10% of trained datam



In [30]:
###Model 0: Getting a baseline - Skleanrs multinomial naive bayes with tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #convert words to numbers
                    ("clf", MultinomialNB()) #model the text

])

#fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
#Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)

In [32]:
baseline_score

0.8044619422572179

In [34]:
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 80.45%


In [36]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:30]

array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1])

In [37]:
### creating an evaluation function for our modeling experiements
#Accuracy, Precision, Recall and f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, preicision, recall and f1-score
  """
  #calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  
  #precision, recall and f1-score(using weighted average)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [38]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 80.4461942257218,
 'f1': 0.7993754401853582,
 'precision': 0.8113250534023555,
 'recall': 0.8044619422572179}

In [55]:
### Model 1: Feed Forward NN

# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"


In [56]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [57]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [58]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [59]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20220725-231623
Epoch 1/5
215/215 [==============================] - 5s 5ms/step - loss: 0.6118 - accuracy: 0.6894 - val_loss: 0.5161 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4424 - accuracy: 0.8187 - val_loss: 0.4420 - val_accuracy: 0.7940
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3482 - accuracy: 0.8583 - val_loss: 0.4304 - val_accuracy: 0.8163
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2851 - accuracy: 0.8904 - val_loss: 0.4391 - val_accuracy: 0.8123
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2372 - accuracy: 0.9110 - val_loss: 0.4604 - val_accuracy: 0.8018


In [60]:
#check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4604 - accuracy: 0.8018


[0.460441917181015, 0.8018372654914856]

In [61]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[4.47615385e-02],
       [6.48659945e-01],
       [9.98234630e-01],
       [1.09312452e-01],
       [9.74001110e-01],
       [5.12504950e-02],
       [6.38423907e-03],
       [1.75525859e-01],
       [9.96820688e-01],
       [9.78685498e-01],
       [2.46737767e-02],
       [1.96991459e-01],
       [9.99553621e-01],
       [4.73157652e-02],
       [5.00250384e-02],
       [5.19756496e-01],
       [9.98041868e-01],
       [2.36845255e-01],
       [5.99966466e-01],
       [4.76100892e-01],
       [1.02969714e-01],
       [3.17111075e-01],
       [9.04221773e-01],
       [5.93289316e-01],
       [3.27259570e-01],
       [9.98970628e-01],
       [2.42150813e-01],
       [2.47893520e-02],
       [9.69103038e-01],
       [9.99899864e-01],
       [9.99446094e-01],
       [1.67539239e-01],
       [4.18343872e-01],
       [3.35347988e-02],
       [6.88077286e-02],
       [9.70032930e-01],
       [7.41307717e-03],
       [4.12475169e-01],
       [3.84077042e-01],
       [3.91551778e-02],


In [62]:
model_1_pred_probs[0]

array([0.04476154], dtype=float32)

In [63]:
model_1_pred_probs.shape

(762, 1)

In [64]:
model_1_pred_probs[:10]

array([[0.04476154],
       [0.64865994],
       [0.9982346 ],
       [0.10931245],
       [0.9740011 ],
       [0.0512505 ],
       [0.00638424],
       [0.17552586],
       [0.9968207 ],
       [0.9786855 ]], dtype=float32)

In [65]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1

In [66]:
#Calculate model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 80.18372703412074,
 'f1': 0.801355032127482,
 'precision': 0.8012130523645551,
 'recall': 0.8018372703412073}

In [69]:
baseline_results = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [68]:
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'me',
 'as',
 'up',
 'just',
 'but',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'its',
 'all',
 'will',
 'after',
 'no',
 'fire',
 'has',
 'an',
 'when',
 'if',
 'get',
 'now',
 'we',
 'about',
 'new',
 'via',
 'more',
 'or',
 'people',
 'what',
 'news',
 'one',
 'how',
 'he',
 'they',
 'over',
 'dont',
 'been',
 'who',
 'into',
 'video',
 'were',
 'do',
 'can',
 '2',
 'us',
 'disaster',
 'emergency',
 'there',
 'police',
 'than',
 'would',
 'still',
 'his',
 'some',
 'her',
 'body',
 'burning',
 'storm',
 'crash',
 'back',
 'had',
 'suicide',
 'man',
 'why',
 'know',
 'got',
 'them',
 'buildings',
 'world',
 'time',
 'off',
 'see',
 'california',
 'first',
 'day',
 'rt',
 'bomb',
 'youtube',
 'cant',
 'love',
 'nuclear',
 'going',
 'their',
 'dead',
 'two',
 'train',
 'killed',
 'go'

In [70]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [78]:
#get weight matrix of embedding layer
#numerical representations of each token in our training data, learned for 5 epochs
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [77]:
embed_weights

TensorShape([1, 10000, 128])

In [80]:
#projector tool
import io
out_v = io.open('vectors.tsv', 'w', encoding="utf-8")
out_m = io.open('metadata.tsv', 'w', encoding="utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue

  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [82]:
#download files from colab and upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>